# LDA에서 Dynamic Topic Model

<img src="4.PNG">
<img src="5.PNG">

- 뭉텅이 하나하나가 LDA 1개
- 알파와 베타 파라미터를 공유(정확히 공유라기보단 오른쪽으로 이동)
- 결국엔 우리가 알고 싶은 건 W -> 토픽들의 단어 분포
- 지금 LDA가 3개 있는데, 이 3개는 DTM이 보고싶은 시간대,시기 3가지
- DTM은 시간의 흐름을 continuous하게 본 게 아니고 discrete하게 봄
    - ex) 작년, 3년 전, 4년 전 (작년걸로 3년 전을, 3년 전걸로 4년 전을), 더 먼 과거에서 과거를 추정하는 느낌
- 결론 : DTM은 시간의 흐름을 특정 단위로 잘라서 보겠다, 그리고 자른 단위만큼 LDA를 돌릴 건데 LDA를 연결해놓은 것

* LDA를 time slice 단위로 여러개를 붙인 형태
    - 토픽을 시간의 변화에 따라 확인할 수 없을까라는 생각으로부터 시작
    - 즉 DPM은 LDA인데 시간이 들어간 것
    - 젤 중요한 점은 시간이 정해진 구간으로 잘려져있다 -> time slice
* 토픽 분포에 해당하는 파라미터인 Beta값을 time slice가 넘어갈때 넘겨주는 것으로 시간대별 토픽 분포를 반영하게 함

<img src="6.PNG">

### 1. 대화내용 불러오기

In [1]:
import pandas as pd
import numpy as np
import pickle
from pprint import pprint
import re

In [16]:
with open("data/cleaned_data.pk", "rb") as f:
    data = pickle.load(f)

data.reset_index(drop=True, inplace=True)
print(data.head())
print(data.info())

             Date User                Message
0  2018. 5. 3. 00  주현                엥 결국 안먹음
1  2018. 5. 3. 00  주현    너 그 야구부 엠티는 가는거야 그래서
2  2018. 5. 3. 00  현영                      안가
3  2018. 5. 3. 00  현영        근엄마한테갈거라고말해놓긴햇는디
4  2018. 5. 3. 00  현영                  보내준다허면
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22914 entries, 0 to 22913
Data columns (total 3 columns):
Date       22914 non-null object
User       22912 non-null object
Message    22914 non-null object
dtypes: object(3)
memory usage: 537.2+ KB
None


Dynamic Topic Model에서 LDA와 ATM의 차이가 어떤게 있을까?
</br></br>
- LDA는 아무것도 없었다고 치면 ATM은 추가된 게 author2doc변수 1개
    - 보고싶었던 게 저자별 문서 정보였으니까
- DTM은 시간대별 무언가 보고싶은 거니까 시간대별로 잘린 정보를 추가해서 줘야해
    - 정보가 추가되면 그게 LAD sequence model! 
    - 그 정보가 어떤 건지 알아보자

### 2. 분석시기 설정하기

In [17]:
# 시간정보 열 datetime 정보로 변환
data['Date'] = pd.to_datetime(data['Date'])

# 인덱스 넣기
data = data.set_index('Date')
data.head()

,User,Message
Date,,
2018-05-03,주현,엥 결국 안먹음
2018-05-03,주현,너 그 야구부 엠티는 가는거야 그래서
2018-05-03,현영,안가
2018-05-03,현영,근엄마한테갈거라고말해놓긴햇는디
2018-05-03,현영,보내준다허면


#### 연도별 대화내용 분리 (DTM 분석을 위해)

In [18]:
# 지정된 시기별로 분리
year2018 = data['2018-05-03' :  '2018-12-31']
year2019 = data['2019-01-01' :  '2019-12-31']
year2020 = data['2020-01-01' :  '2020-06-17']

In [19]:
print(year2018.head())
print(year2019.head())
print(year2020.head())

           User                Message
Date                                  
2018-05-03  주현                엥 결국 안먹음
2018-05-03  주현    너 그 야구부 엠티는 가는거야 그래서
2018-05-03  현영                      안가
2018-05-03  현영        근엄마한테갈거라고말해놓긴햇는디
2018-05-03  현영                  보내준다허면
           User                Message
Date                                  
2019-01-01  지이    얘두라 따랑해 새해복 많이받아❤️☺️
2019-01-01  현영             사랑해 민나상 모두들
2019-01-01  현영                    오메데또
2019-01-01  현영               앗 언니자화상이다
2019-01-01  지이                     졸귀탱
                    User        Message
Date                                   
2020-01-01 00:00:00  현영            고구마 
2020-01-15 18:00:00  영현             늼들아
2020-01-15 18:00:00  영현    우리 설끝나고 모이자 
2020-01-15 18:00:00  지의             그러자
2020-01-15 18:00:00  지의              제발


In [20]:
print(year2018.info())
print(year2019.info())
print(year2020.info())

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5991 entries, 2018-05-03 00:00:00 to 2018-12-31 23:00:00
Data columns (total 2 columns):
User       5991 non-null object
Message    5991 non-null object
dtypes: object(2)
memory usage: 140.4+ KB
None
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 13858 entries, 2019-01-01 00:00:00 to 2019-12-31 23:00:00
Data columns (total 2 columns):
User       13856 non-null object
Message    13858 non-null object
dtypes: object(2)
memory usage: 324.8+ KB
None
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3065 entries, 2020-01-01 00:00:00 to 2020-06-17 17:00:00
Data columns (total 2 columns):
User       3065 non-null object
Message    3065 non-null object
dtypes: object(2)
memory usage: 71.8+ KB
None


##### 시기별로 나눠서 데이터 저장하기

In [21]:
# time_slice별로 데이터 갯수를 저장할 리스트 생성
time_slice = [] 
# time_slice: 각 slice별로 문서가 나눠줬다는 거 알려주는 리스트
# 어떤식으로 gensim에서 구분하냐 -> time_slice의 가정 : 시간대별로 list에 순서대로 들어가있다
# 즉 어디서부터 어디까지 첫번재 time_slice인지 알려주는 리스트다

# 시기별 데이터 가져오기
slice0 = list(year2018['Message'])
slice1 = list(year2019['Message'])
slice2 = list(year2020['Message'])

# LDA 분석할 때와 비슷하게 모든 분석 대상 텍스트를 담는 리스트 생성
tokenized_data = [msg.split() for msg in (slice0 + slice1 + slice2)]

# 각 slice에 들어 있는 갯수를 원소로 갖는 리스트 생성
time_slice.append(len(slice0))
time_slice.append(len(slice1))
time_slice.append(len(slice2))

In [22]:
print(len(tokenized_data))
print(time_slice)
print(slice0[:5])
print(slice1[:5])
print(slice2[:5])
pprint(tokenized_data[:5])

22914
[5991, 13858, 3065]
[' 엥 결국 안먹음', ' 너 그 야구부 엠티는 가는거야 그래서', ' 안가', ' 근엄마한테갈거라고말해놓긴햇는디', ' 보내준다허면']
[' 얘두라 따랑해 새해복 많이받아❤️☺️', ' 사랑해 민나상 모두들', ' 오메데또', ' 앗 언니자화상이다', ' 졸귀탱']
[' 고구마 ', ' 늼들아', ' 우리 설끝나고 모이자 ', ' 그러자', ' 제발']
[['엥', '결국', '안먹음'],
 ['너', '그', '야구부', '엠티는', '가는거야', '그래서'],
 ['안가'],
 ['근엄마한테갈거라고말해놓긴햇는디'],
 ['보내준다허면']]


### 3. Dynamic Topic Model 돌리기

In [23]:
from gensim.models import ldaseqmodel
from gensim.corpora import Dictionary, bleicorpus
from gensim import corpora

import os

#### dictionary와 corpus 만들기

In [26]:
# DTM dictionary 저장.
if not os.path.exists('kakao(DTM)_dict'):
    dictionary = corpora.Dictionary(tokenized_data)
    dictionary.save('kakao(DTM)_dict')
else:
    dictionary = Dictionary.load('kakao(DTM)_dict')

# DTM Corpus 저장.
if not os.path.exists('kakao(DTM)_corpus'):
    corpus = [dictionary.doc2bow(doc) for doc in tokenized_data]
    corpora.BleiCorpus.serialize('kakao(DTM)_corpus', corpus)
else:
    corpus = bleicorpus.BleiCorpus('kakao(DTM)_corpus')

### Run DTM model 

In [ ]:
# 주의! 매우매우매우 오래걸릴지도 모릅니다... 시간과 여유가 있을때 해보세요!
NUM_TOPICS = 4

if not os.path.exists('kakao(DTM)_model'):
    dtm_model = ldaseqmodel.LdaSeqModel(corpus=corpus,
                                       id2word=dictionary,
                                       time_slice=time_slice,
                                       num_topics=NUM_TOPICS,
                                       passes=10) # 이것도 1000정도 넘게 해줘야 성능 good
    dtm_model.save('kakao(DTM)_model')
else:
    dtm_model = ldaseqmodel.LdaSeqModel.load('kakao(DTM)_model')

##### DTM 결과 보기

In [ ]:
# 고정된 시간 내에서 전체 토픽보기
pprint(dtm_model.print_topics(time=0, top_terms=10))

# 이 모델을 통해 알 수 있는 점
# 시간이 지나도 사람들이 쓰는 단어가 비슷